In [1]:
import pandas as pd
import numpy as np

## 5. PCA

АЛГОРИТМ РЕАЛИЗАЦИИ PCA
1. Стандартизировать данные.
2. Рассчитать ковариационную матрицу для объектов.
3. Рассчитать собственные значения и собственные векторы для ковариационной матрицы.
4. Отсортировать собственные значения и соответствующие им собственные векторы.
5. Выбрать  наибольших собственных значений и сформировать матрицу соответствующих собственных векторов.
6. Преобразовать исходные данные, умножив матрицу данных на матрицу отобранных собственных векторов.

Рассмотрим все шаги на примере.

>Примечание. Так как в процессе реализации алгоритма мы будем сталкиваться с достаточно громоздкими вычислениями, часть из них мы будем сопровождать кодом для их выполнения в Python.>

Допустим, у нас есть набор данных, состоящий из четырёх признаков:

$x_1$|$x_2$|$x_3$|$x_4$
-|-|-|-
1|2|3|4
5|5|6|7
1|4|2|3
5|3|2|1
8|1|2|2


Мы хотим сократить количество измерений до двух.



In [4]:
import numpy as np
import pandas as pd
A = np.matrix([[1,2,3,4],
               [5,5,6,7],
               [1,4,2,3],
               [5,3,2,1],
               [8,1,2,2]])

df = pd.DataFrame(A,columns  = ['x1','x2','x3','x4'])
# Стандартизировать данные.
df_std  = (df - df.mean()) / (df.std())
df_std

,x1,x2,x3,x4
0,-1.000000,-0.632456,0.000000,0.260623
1,0.333333,1.264911,1.732051,1.563740
2,-1.000000,0.632456,-0.577350,-0.173749
3,0.333333,0.000000,-0.577350,-1.042493
4,1.333333,-1.264911,-0.577350,-0.608121


In [5]:
# Рассчитать ковариационную матрицу для объектов.
cov_mat = np.cov(df_std.T)
cov_mat

array([[ 1.        , -0.31622777,  0.04811252, -0.18098843],
       [-0.31622777,  1.        ,  0.63900965,  0.61812254],
       [ 0.04811252,  0.63900965,  1.        ,  0.94044349],
       [-0.18098843,  0.61812254,  0.94044349,  1.        ]])

In [6]:
# Рассчитать собственные значения и собственные векторы для ковариационной матрицы.
eigen_val, eigen_vectors = np.linalg.eig(cov_mat)

Теперь нам необходимо выбрать два собственных вектора, так как в итоге мы хотим получить две компоненты. Нужно взять векторы с наибольшими собственными значениями. При нахождении собственных векторов в Python они выводятся в соответствии с убыванием соответствующих собственных значений, поэтому берём первые два вектора:   

$$e_1 \ (0.161960, -0.524048, -0.585896, -0.596547)$$
$$e_2 \ (-0.917059, 0.206922, -0.320539, -0.115935)$$
   
Осталось только найти главные компоненты. Для этого перемножаем матрицу стандартизированных признаков на матрицу с собственными векторами и получаем матрицу главных компонент:   
   
   
$$\left(\begin{array}{rrrr}-1.000000 & -0.632456 & 0.000000 & 0.260623 \\ 0.333333 & 1.264911 & 1.732051 & 1.563740 \\ -1.000000 & 0.632456 & -0.577350 & -0.173749 \\ 0.333333 & 0.000000 & -0.577350 & -1.042493 \\ 1.333333 & -1.264911 & -0.577350 & -0.608121\end{array}\right) \cdot \left(\begin{array}{rr}0.161960 & -0.917059 \\ -0.524048 & 0.206922 \\ -0.585896 & -0.320539 \\ -0.596547 & -0.115935\end{array}\right)=\left(\begin{array}{rr}0.014003 & 0.755975 \\ -2.556534 & -0.780432 \\ -0.051480 & 1.253135 \\ 1.014150 & 0.000239 \\ 1.579861 & -1.228917\end{array}\right)$$

In [15]:
df_std @ eigen_vectors[:,:2]

,0,1
0,0.014003,0.755975
1,-2.556534,-0.780432
2,-0.051480,1.253135
3,1.014150,0.000239
4,1.579861,-1.228917


Таким образом, мы получили две главных компоненты. Можно проверить, получили ли бы мы тот же результат, если бы воспользовались сразу готовым алгоритмом на `Python` — `PCA`.

В параметр `n_components` в качестве значения можно передать количество необходимых компонентов или минимально допустимую объяснённую дисперсию в виде десятичной дроби. Например, если нам нужно столько компонент, чтобы они объясняли не менее 90 % разброса данных, то мы запишем `n_components = 0.9`.

In [18]:
#импортируем нужный алгоритм
from sklearn.decomposition import PCA
#определяем метод главных компонент с двумя компонентами
pca = PCA(n_components=2)
#обучаем алгоритм на наших данных
principalComponents = pca.fit_transform(df_std)
principalComponents

array([[-1.40033078e-02,  7.55974765e-01],
       [ 2.55653399e+00, -7.80431775e-01],
       [ 5.14801919e-02,  1.25313470e+00],
       [-1.01415002e+00,  2.38808310e-04],
       [-1.57986086e+00, -1.22891650e+00]])

### Задание 5.1
Найдите матрицу ковариаций для векторов $(3, 4, 1)$ и $(1, 6, 2)$. В качестве ответа укажите сумму всех значений матрицы, округлённую до двух знаков после точки-разделителя.

In [29]:
ans = np.cov(np.array([[3,4,1],[1,6,2]])).sum()
round(ans,2)

14.33

### Задание 5.5
Дана матрица признаков:
```python
A = np.matrix([[8,7,2,9],
               [1,3,6,3],
               [7,2,0,3],
               [10,3,1,1],
               [8,1,3,4]])
```
Какое минимальное количество главных компонент надо выделить, чтобы сохранить информацию о как минимум 90 % разброса данных?

In [58]:
A = np.matrix([[8,7,2,9],
               [1,3,6,3],
               [7,2,0,3],
               [10,3,1,1],
               [8,1,3,4]])
pca = PCA(n_components=0.9)
#обучаем алгоритм на наших данных
A_std = (A - A.mean(axis=0))/A.std(axis=0,ddof=1)
pca.fit_transform(A_std).shape[1]

3

## 6. Снижение размерности: SVD и t-SNE

Начнём с сингулярного разложения. SVD используется не только для снижения размерности, но и в целом имеет множество применений в машинном обучении, например, в рекомендательных системах — в одном из следующих модулей вы сможете изучить эту тему подробнее.

Суть сингулярного разложения заключается в следующей теореме ↓

Любую прямоугольную матрицу $A$ размера $(n, m)$ можно представить в виде произведения трёх матриц:

$$A_{n \times m}=U_{n \times n} \cdot D_{n \times m} \cdot V_{m \times m}^{T}$$

В этой формуле:

$U$ — матрица размера $(n, n)$. Все её столбцы ортогональны друг другу и имеют единичную длину. Такие матрицы называются ортогональными. Эта матрица содержит нормированные собственные векторы матрицы $AA^T$.  
$D$  — матрица размера $(n, m)$. На её главной диагонали стоят числа, называемые сингулярными числами (они являются корнями из собственных значений матриц $AA^T$ и $A^AT$), а вне главной диагонали стоят нули. Если мы решаем задачу снижения размерности, то элементы этой матрицы, если их возвести в квадрат, можно интерпретировать как дисперсию, которую объясняет каждая компонента.  
$V$ — матрица размера $(m, m)$. Она тоже ортогональная и содержит нормированные собственные векторы матрицы $A^AT$.  

Допустим, нам необходимо получить сингулярное разложение для следующей матрицы:

$$A=\left(\begin{array}{ccc}-1 & 1 & 0 \\ -1 & -1 & 1\end{array}\right)$$

Начнём с поиска матрицы $U$.    
Для этого нам необходимо сначала вычислить произведение матриц $A$ и $A^T$:

$$A \cdot A^{T}=\left(\begin{array}{rrr}-1 & 1 & 0 \\ -1 & -1 & 1\end{array}\right) \cdot\left(\begin{array}{rr}-1 & -1 \\ 1 & -1 \\ 0 & 1\end{array}\right)$$

### Задание 6.1
Вычислите получившееся произведение матриц  и $A \cdot A^{T}$:



In [68]:
A = np.array([[-1,1,0],[-1,-1,1]])
AAT = A@A.T

array([[2, 0],
       [0, 3]])

Теперь нам необходимо решить характеристическое уравнение $det(A - \lambda E) = 0$:

In [104]:
from sympy import *

lamda = symbols('lamda')
p = Matrix(AAT).charpoly(lamda)
factor(p.as_expr())
solve(p,lamda)

[2, 3]

Получаем два собственных значения: $2, 3$ . Теперь можно извлечь из них корень и получить сингулярные значения (они нужны для матрицы $D$):
$$\sigma_1 = \sqrt{2}, \ \sigma_2 = \sqrt{3}$$

Найдем собственные вектора и нормируем их:, чтобы составить матрицу $U$.

In [159]:
AAT = A@A.T
eigenvec = Matrix(AAT).eigenvects()
[display(x[2][0]) for x in eigenvec];
[display(np.array(x[2][0])/np.linalg.norm(np.array(x[2][0],dtype=float))) for x in eigenvec];

Matrix([
[1],
[0]])

Matrix([
[0],
[1]])

array([[1.00000000000000],
       [0]], dtype=object)

array([[0],
       [1.00000000000000]], dtype=object)

для нахождения матрицы V нам необходимо повторить всё для $A^{T} \cdot A$

In [126]:
ATA = A.T@A
lamda = symbols('lamda')
p = Matrix(ATA).charpoly(lamda)
factor(p.as_expr())
solve(p,lamda)

[0, 2, 3]

$$\sigma_1 = 0, \ \sigma_2 = \sqrt{2}, \ \sigma_3 = \sqrt{3}$$

In [160]:
ATA = A.T@A
eigenvec = Matrix(ATA).eigenvects()
[display(x[2][0]) for x in eigenvec];
[display(np.array(x[2][0])/np.linalg.norm(np.array(x[2][0],dtype=float))) for x in eigenvec];

Matrix([
[1/2],
[1/2],
[  1]])

Matrix([
[-1],
[ 1],
[ 0]])

Matrix([
[-1],
[-1],
[ 1]])

array([[0.408248290463863],
       [0.408248290463863],
       [0.816496580927726]], dtype=object)

array([[-0.707106781186547],
       [0.707106781186547],
       [0]], dtype=object)

array([[-0.577350269189626],
       [-0.577350269189626],
       [0.577350269189626]], dtype=object)

Теперь мы можем записать соответствующую матрицу $V$. Для этого полученные собственные векторы располагаем в столбцах матрицы в порядке убывания их собственных чисел:

In [ ]:
V = 

Для того чтобы реализовать сингулярное разложение с помощью библиотеки `sklearn`, необходимо использовать алгоритм `TruncatedSVD()`, в который передаётся n_components в качестве параметра, определяющего количество итоговых компонент:
```python
# создаём объект класса TruncatedSVD
# n_components — размерность нового пространства, n_iter — количество итераций
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
# обучаем модель на данных X
svd.fit(X)
# применяем уменьшение размерности к матрице X
transformed = svd.transform(data)
```

In [165]:
from sklearn.decomposition import TruncatedSVD
# создаём объект класса TruncatedSVD
# n_components — размерность нового пространства, n_iter — количество итераций
svd = TruncatedSVD(n_components=2, n_iter=7, random_state=42)
# обучаем модель на данных X
svd.fit(A)
# применяем уменьшение размерности к матрице X
transformed = svd.transform(A)
transformed

array([[-2.22044605e-16,  1.41421356e+00],
       [ 1.73205081e+00, -1.94289029e-16]])

## t-SNE
Итак, вы уже знакомы с двумя алгоритмами для понижения размерности — PCA и SVD. Теперь давайте познакомимся с третьим — t-SNE (стохастическое вложение соседей с t-распределением). Его преимущество относительно первых двух заключается в том, что он может реализовывать уменьшение размерности и разделение для данных, которые являются линейно неразделимыми. Подобные данные можно визуализировать так:

Для реализации t-SNE в sklearn понадобится алгоритм TSNE():
```python
# импортируем класс TSNE из модуля manifold библиотеки sklearn
from sklearn.manifold import TSNE
# создаём объект класса TSNE
# n_components — размерность нового пространства
tsne = TSNE(n_components=2, perplexity=30, n_iter=500, random_state=42)
# обучаем модель на данных X
tsne.fit(X)
# применяем уменьшение размерности к матрице X
tsne.transform(X)
```

In [168]:
# импортируем класс TSNE из модуля manifold библиотеки sklearn
from sklearn.manifold import TSNE
# создаём объект класса TSNE
# n_components — размерность нового пространства
tsne = TSNE(n_components=2, perplexity=30, n_iter=500, random_state=42)
# обучаем модель на данных X
# tsne.fit(A)
# применяем уменьшение размерности к матрице X
# tsne.transform(A)
tsne.fit_transform(A)

array([[  164.85704,  1814.0579 ],
       [ -164.85692, -1814.0576 ]], dtype=float32)

# ==========================================

In [66]:
# x = symbols('x')
# f = -x**4 + 6*x**3 -4*x**2 + 80
# f_x = f.diff(x)
# solve(f_x,x)

[0, 1/2, 4]

In [84]:
lamda = symbols('lamda')
p = Matrix(AAT).charpoly(lamda)
factor(p.as_expr())


(lamda - 3)*(lamda - 2)

In [88]:
# lamda = symbols('lamda')
# p = Matrix(AAT).charpoly(lamda)
# factor(p.as_expr())
# solve(p,lamda)

[2, 3]